# ONS population projections

The UK Office of National Statistics produce national population projections ([overview](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationprojections/compendium/nationalpopulationprojections/)) and each of the England, Wales, Scotland and Northern Ireland produce sub-national projections at the local area scale.

As of April 2018, the latest available sub-national projections are based on 2014 mid-year estimates. Mid-year estimates are produced annually; the most recent census was in 2011.

ONS Methodology papers:
- [National population projections > Background and methodology](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationprojections/compendium/nationalpopulationprojections/2016basedprojections/backgroundandmethodology)
- [Methodology used to produce the 2014-based subnational population projections for England](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationprojections/methodologies/methodologyusedtoproducethesubnationalpopulationprojectionsforengland)

This notebook:
1. Downloads data from each national statistics office
2. Compiles a single [tidy](http://vita.had.co.nz/papers/tidy-data.html) data frame of projected population
3. Outputs to CSV
4. Outputs to multiple CSVs for use in [smif](smif.readthedocs.io/en/latest/)

Requirements:
- [ukcensusapi ](https://github.com/virgesmith/UKCensusAPI)
  - `pip install git+https://github.com/virgesmith/UKCensusAPI.git`
- [requests](http://docs.python-requests.org/en/master/), [pandas](http://pandas.pydata.org/)
  - `pip install requests pandas`
- [jupyter](http://jupyter.org/) (to run this notebook): 
  - `pip install jupyter`

## 1. Download data

In [ ]:
import os
import json
import zipfile

import pandas as pd
import requests

In [ ]:
def download(url, file_path):
    """Download a file from a URL to a given path
    """
    if os.path.exists(file_path):
        print("Already have", file_path)
    else:
        response = requests.get(url)
        with open(file_path, 'wb') as fd:
            for chunk in response.iter_content(chunk_size=1024):
                fd.write(chunk)
        print("Downloaded", file_path)

### England
- summary: [Subnational Population Projections, Local Authorities in England: SNPP Z1](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationprojections/datasets/localauthoritiesinenglandz1)
- data: https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationprojections/datasets/localauthoritiesinenglandz1/2014based/snppz1population.zip
- license: Open Government Licence http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/

To use the `ukcensusapi` package it is recommended to register with Nomis and store an API key in the NOMIS_WEB_API environment variable as described in the [UKCensusAPI docs](https://github.com/virgesmith/UKCensusAPI#api-key). Once registered with Nomis, you should find your API key on [this page](https://www.nomisweb.co.uk/myaccount/webservice.asp).

In [ ]:
import ukcensusapi.Nomisweb as Api

cache_dir = "raw_data"
data_api = Api.Nomisweb(cache_dir)

In [ ]:
table_internal = "NM_2006_1" # 2014-based SNPP
query_params = {
    "gender": "0",
    "c_age": "200",
    "MEASURES": "20100",
    "date": "latest", # 2014-based
    "projected_year": "2014...2039",
    "select": "geography_code,geography_name,projected_year_name,obs_value",
    "geography": "1946157057...1946157382"
}
eng_snpp = data_api.get_data(table_internal, query_params)

- summary: [Experimental variant 2014-based subnational population projections for local authorities in England, mid-2014 to mid-2039 – high fertility](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationprojections/adhocs/005982variant2014basedsubnationalpopulationprojectionsforlocalauthoritiesinenglandmid2014tomid2039highfertility)

In [ ]:
download(
    "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationprojections/adhocs/005982variant2014basedsubnationalpopulationprojectionsforlocalauthoritiesinenglandmid2014tomid2039highfertility/2014snpphighfertilityvariant.xls",
    os.path.join("raw_data", "snpp_england_high_fertility.xls")
)

- summary: [Experimental variant 2014-based subnational population projections for local authorities in England, mid-2014 to mid-2039 – zero net migration](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationprojections/adhocs/005983variant2014basedsubnationalpopulationprojectionsforlocalauthoritiesinenglandmid2014tomid2039zeronetmigration)

In [ ]:
download(
    "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationprojections/adhocs/005983variant2014basedsubnationalpopulationprojectionsforlocalauthoritiesinenglandmid2014tomid2039zeronetmigration/2014snppzeronetmigrationvariant.xls",
    os.path.join("raw_data", "snpp_england_zero_net_migration.xls")
)

### Wales
- summary: [Population > Projections > Local authority > 2014-based](https://statswales.gov.wales/Catalogue/Population-and-Migration/Population/Projections/Local-Authority/2014-based)
- data: http://open.statswales.gov.wales/dataset/popu5099
- license: Open Government License / Crown Copyright 2017

In [ ]:
if os.path.exists("raw_data/snpp_wales.csv"):
    print("Already have snpp_wales.csv")
else:
    fields = ['Area_AltCode1','Area_ItemName_ENG','Variant_Code','Year_Code','Data','Gender_Code','Age_Code','Area_Hierarchy']
    # StatsWales is an OData endpoint, so select fields of interest
    url = "http://open.statswales.gov.wales/dataset/popu5099?$select={}".format(",".join(fields))
    # use OData syntax to filter P (persons), AllAges (all ages), Area_Hierarchy 596 (LADs)
    url += "&$filter=Gender_Code eq 'P' and Age_Code eq 'AllAges' and Area_Hierarchy eq 596"
    data = []
    while True:
        print(url)
        r = requests.get(url)
        r_data = r.json()
        data += r_data['value']
        if "odata.nextLink" in r_data:
            url = r_data["odata.nextLink"]
        else:
            break
    df = pd.DataFrame(data)
    df.to_csv('raw_data/snpp_wales.csv', index=False)

### Scotland
- summary: [Population Projections for Scottish Areas (2014-based)](https://www.nrscotland.gov.uk/statistics-and-data/statistics/statistics-by-theme/population/population-projections/sub-national-population-projections/2014-based/list-of-tables)
- data: https://www.nrscotland.gov.uk/files//statistics/population-projections/snpp-2014/pop-proj-scot-areas-14-populationvariants.csv
- license: Open Government Licence / Crown Copyright 2014 Data supplied by National Records of Scotland

In [ ]:
download(
    "https://www.nrscotland.gov.uk/files//statistics/population-projections/snpp-2014/pop-proj-scot-areas-14-populationvariants.csv",
    os.path.join("raw_data", "snpp_scotland.csv")
)

- summary: [Local Authority Districts (December 2016) Names and Codes in the United Kingdom](http://geoportal.statistics.gov.uk/datasets/local-authority-districts-december-2016-names-and-codes-in-the-united-kingdom)
- data: https://opendata.arcgis.com/datasets/464be6191a434a91a5fa2f52c7433333_0.csv
- license: OGL http://www.nationalarchives.gov.uk/doc/open-government-licence/, Contains National Statistics data © Crown copyright and database right [2016]

In [ ]:
download(
    "https://opendata.arcgis.com/datasets/464be6191a434a91a5fa2f52c7433333_0.csv",
    os.path.join("raw_data", "lad_names_codes.csv")
)

### Northern Ireland
- summary: [2014-based Population Projections for Areas within Northern Ireland](https://www.nisra.gov.uk/publications/2014-based-population-projections-areas-within-northern-ireland)
- data: https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/SNPP14-LGD14-Tot-1439.xlsx
- license: Open Government Licence / Crown Copyright 2018 Source: NISRA : Website: www.nisra.gov.uk

In [ ]:
download(
    "https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/SNPP14-LGD14-Tot-1439.xlsx",
    os.path.join("raw_data", "snpp_northern_ireland.xlsx")
)

- summary: [Z6 - Zipped population projections data files, Northern Ireland](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationprojections/datasets/z6zippedpopulationprojectionsdatafilesnorthernireland)
- data: https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationprojections/datasets/z6zippedpopulationprojectionsdatafilesnorthernireland/2014based/rft-table-z6-open-data---northern-ireland.zip
- license: OGL

In [ ]:
download(
    "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationprojections/datasets/z6zippedpopulationprojectionsdatafilesnorthernireland/2014based/rft-table-z6-open-data---northern-ireland.zip",
    os.path.join("raw_data", "npp_northern_ireland.zip")
)

In [ ]:
with zipfile.ZipFile(os.path.join("raw_data", "npp_northern_ireland.zip"), 'r') as zf:
    zf.extractall("raw_data")

## 2. Compile data

### England

In [ ]:
eng_snpp

In [ ]:
england_df = eng_snpp.rename(
    columns={
        "GEOGRAPHY_CODE": "lad_code",
        "GEOGRAPHY_NAME": "lad_name",
        "PROJECTED_YEAR_NAME": "year",
        "OBS_VALUE": "population"
    }
).groupby(
    ['lad_code', 'year']).sum().reset_index().sort_values(
    by=['year', 'lad_code']
)
england_df['scenario'] = 'principal'
england_df

In [ ]:
england_hf_df = pd.read_excel(
    "raw_data/snpp_england_high_fertility.xls",
    sheet_name="Persons"
).drop(
    columns=['AGE GROUP', 'SEX']
).groupby(
    ['AREA CODE', 'AREA NAME']
).sum().reset_index().melt(
    id_vars=['AREA CODE', 'AREA NAME'],
    var_name='year',
    value_name='population'
).rename(
    columns={
        "AREA CODE": "lad_code",
        "AREA NAME": "lad_name"
    }
)
england_hf_df.population = england_hf_df.population.astype(int)
england_hf_df['scenario'] = 'high_fertility'
england_hf_df

In [ ]:
england_znm_df = pd.read_excel(
    "raw_data/snpp_england_zero_net_migration.xls",
    sheet_name="Persons"
).drop(
    columns=['AGE GROUP', 'SEX']
).groupby(
    ['AREA CODE', 'AREA NAME']
).sum().reset_index().melt(
    id_vars=['AREA CODE', 'AREA NAME'],
    var_name='year',
    value_name='population'
).rename(
    columns={
        "AREA CODE": "lad_code",
        "AREA NAME": "lad_name"
    }
)
england_znm_df['scenario'] = 'zero_net_migration'
england_znm_df

### Northern Ireland

In [ ]:
ni_df = pd.read_excel(
    "raw_data/snpp_northern_ireland.xlsx",
    index_col=[0,1],
    header=2
).dropna(
)[1:12].reset_index(
).rename(
    columns={
        'level_0': 'lad_name',
        'level_1': 'lad_code'
    }
).melt(
    id_vars=['lad_name', 'lad_code'],
    var_name='year',
    value_name='principal'
)
ni_df

In [ ]:
from bs4 import BeautifulSoup

def read_excel_xml(path, sheet_name):
    file = open(path).read()
    soup = BeautifulSoup(file,'xml')
    worksheet = []
    for sheet in soup.findAll('Worksheet'): 
        if sheet["ss:Name"] == sheet_name:
            for row in sheet.findAll('Row'):
                row_as_list = []
                for cell in row.findAll('Cell'):
                    data = cell.find('Data')
                    if data:
                        row_as_list.append(data.text)

                worksheet.append(row_as_list)
    return worksheet

In [ ]:
ni_hpp_list = read_excel_xml("raw_data/ni_hpp_opendata2014.xml", "Population")

In [ ]:
ni_hpp = pd.DataFrame(ni_hpp_list[1:], columns=ni_hpp_list[0])
ni_hpp['scenario'] = 'high_fertility'
ni_hpp = ni_hpp.drop(
    columns=['Sex', 'Age']
).melt(
    id_vars=['scenario'],
    var_name='year',
    value_name='high_fertility_total'
)
ni_hpp.high_fertility_total = ni_hpp.high_fertility_total.astype(int)
ni_hpp.year = ni_hpp.year.astype(int)
ni_hpp = ni_hpp.drop(columns=['scenario']).groupby(
    'year'
).sum().reset_index()
ni_hpp.head()

In [ ]:
ni_ppp_list = read_excel_xml("raw_data/ni_ppp_opendata2014.xml", "Population")

In [ ]:
ni_ppp = pd.DataFrame(ni_ppp_list[1:], columns=ni_ppp_list[0])
ni_ppp['scenario'] = 'principal'
ni_ppp = ni_ppp.drop(
    columns=['Sex', 'Age']
).melt(
    id_vars=['scenario'],
    var_name='year',
    value_name='principal_total'
)
ni_ppp.principal_total = ni_ppp.principal_total.astype(int)
ni_ppp.year = ni_ppp.year.astype(int)
ni_ppp = ni_ppp.drop(columns=['scenario']).groupby(
    'year'
).sum().reset_index()
ni_ppp = ni_ppp[ni_ppp.year < 2040]
ni_ppp.head()

In [ ]:
ni_ppz_list = read_excel_xml("raw_data/ni_ppz_opendata2014.xml", "Population")

In [ ]:
ni_ppz = pd.DataFrame(ni_ppz_list[1:], columns=ni_ppz_list[0])
ni_ppz['scenario'] = 'zero_net_migration'
ni_ppz = ni_ppz.drop(
    columns=['Sex', 'Age']
).melt(
    id_vars=['scenario'],
    var_name='year',
    value_name='zero_net_migration_total'
)
ni_ppz.zero_net_migration_total = ni_ppz.zero_net_migration_total.astype(int)
ni_ppz.year = ni_ppz.year.astype(int)
ni_ppz = ni_ppz.drop(columns=['scenario']).groupby(
    'year'
).sum().reset_index()
ni_ppz.head()

**NB** Northern Ireland only produces the 'principal' variant population projections, so scale those figures to fit the Northern Ireland country-level projections for the common variants. This is not likely to be consistent with the cohort evolution methodology but does give some variation around the principal projection to use in our scenarios.

In [ ]:
ni_lad = pd.merge(
    ni_df, pd.merge(
        ni_ppz,
        pd.merge(
            ni_hpp,
            ni_ppp,
            how='outer',
            on='year'
        ),
        how='outer',
        on='year'
    ),
    how='outer',
    on='year'
)
ni_lad['zero_net_migration'] = ni_lad.principal * ni_lad.zero_net_migration_total / ni_lad.principal_total
ni_lad['high_fertility'] = ni_lad.principal * ni_lad.high_fertility_total / ni_lad.principal_total
ni_lad_tidy = ni_lad.drop(
    columns=['zero_net_migration_total', 'principal_total', 'high_fertility_total']
).melt(
    id_vars=['lad_name', 'lad_code', 'year'],
    var_name='scenario',
    value_name='population'
)
ni_lad_tidy

### Wales

In [ ]:
wales_df = pd.read_csv(
    "raw_data/snpp_wales.csv",
    dtype={
        'Age_Code': 'str',
        'Variant_ItemNotes_ENG': 'str'
    }
).drop(
    columns=[
        'Age_Code', 'Area_Hierarchy', 'Gender_Code'
    ]
).rename(
    columns={
        "Area_ItemName_ENG": "lad_name",
        "Area_AltCode1": "lad_code",
        "Data": "population",
        "Variant_Code": "scenario",
        "Year_Code": "year"
    }
).sort_values(
    by=['scenario', 'lad_code', 'year']
)
wales_df.population = wales_df.population.astype(int)
wales_df

In [ ]:
def normalise_variant_name(name):
    lu = {
        'High': 'high', 
        'Low': 'low', 
        'Principal': 'principal', 
        'TenYear': 'ten_year_average_migration', 
        'ZeroMig': 'zero_net_migration'
    }
    if name in lu:
        name = lu[name]
    return name
wales_df.scenario = wales_df.scenario.apply(normalise_variant_name)

In [ ]:
wales_df.scenario.unique()

In [ ]:
wales_df

### Scotland

In [ ]:
lad_names_codes = pd.read_csv("raw_data/lad_names_codes.csv")
lad_names_codes

In [ ]:
scotland_df = pd.read_csv(
    "raw_data/snpp_scotland.csv"
).query(
    "area_type == 'Council area'"
).drop(    
    columns=['area_type']
).melt(
    id_vars=['area', 'year'],
    var_name='scenario',
    value_name='population'    
).rename(
    columns={'area': 'lad_name'}
)

# lowercase scenarios
scotland_df.scenario = scotland_df.scenario.apply(lambda s: s.lower())
scotland_df.scenario = scotland_df.scenario.apply(lambda n: n.replace('zero_migration', 'zero_net_migration'))
# look up lad_codes
scotland_df = pd.merge(
    scotland_df, lad_names_codes, 
    how='left', left_on='lad_name', right_on='LAD16NM'
).drop(
    columns=['LAD16CDO', 'LAD16NM', 'FID']
).rename(
    columns={'LAD16CD': 'lad_code'}
)

scotland_df

In [ ]:
scotland_df.scenario.unique()

### Combine common scenarios (variants)

**NB** Wales produces a 'high' variant but not a 'high_fertility' variant - use this as substitute for the purposes of initial data exploration.

In [ ]:
wales_df.scenario = wales_df.scenario.apply(lambda n: n.replace('high', 'high_fertility'))
wales_df

In [ ]:
uk = pd.concat([
    england_df,
    england_hf_df,
    england_znm_df,
    wales_df[wales_df.scenario.isin(('principal', 'high_fertility', 'zero_net_migration'))],
    scotland_df[scotland_df.scenario.isin(('principal', 'high_fertility', 'zero_net_migration'))],
    ni_lad_tidy
])
uk

Output single 'tidy' file for UK.

In [ ]:
uk.to_csv('data/uk_snpp.csv', index=False)

Output one file per scenario

In [ ]:
uk['interval'] = 1
for scenario in ('high_fertility', 'principal', 'zero_net_migration'):
    tmp_df = uk[uk.scenario == scenario].drop(
        columns=['scenario']
    ).rename(
        columns={
            'population': 'value',
            'lad_code': 'region',
            'lad_name': 'location_name'
        }
    )
    tmp_df.to_csv("data/uk_snpp_{}.csv".format(scenario), index=False)
    print("Output {} rows for {}".format(len(tmp_df), scenario))